#AFTER WE HAVE DONE BASIC DATA CLEANING IN EDA FILE WE CAN PERFORM DATA TRANSFORMATION

:

# TEXT PREPROCESSING AND DOWNSAMPLING

##Downsampling the data

Since our data has Credit reporting as majority class . This could make model more bias towards credit reporting.So we will downsample the data.


We will split the original data into training and test sets. This ensures that the test set remains representative of the original data distribution and avoids information leakage.
And then perform downsampling only on the training data to balance the class distribution.

test data will reflect the original class distribution, which might mean it contains more examples from the majority class. This is typically desired because it ensures that your test set is representative of the real-world scenario you are trying to model.

### We will use RandomUnderSampler

The RandomUnderSampler from the imblearn (imbalanced-learn) library is a technique used to handle class imbalance by randomly undersampling the majority class(es) to balance the class distribution in the dataset.

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
X = df_cleaned['Consumer Complaint']
y = df_cleaned['Product']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_data = pd.DataFrame({'text': X_train, 'label': y_train})
            desired_samples_train = {
                'Credit Reporting/Repair Services/Consumer Reports': 300000,
                'Debt Collection': len(train_data[train_data['label'] == 'Debt Collection']),
                'Credit/Prepaid Cards': len(train_data[train_data['label'] == 'Credit/Prepaid Cards']),
                'Mortgage': len(train_data[train_data['label'] == 'Mortgage']),
                'Bank Account Services': len(train_data[train_data['label'] == 'Bank Account Services']),
                'Student loan': len(train_data[train_data['label'] == 'Student loan']),
                'Money Services/Transfer': len(train_data[train_data['label'] == 'Money Services/Transfer']),
                'Personal Loan': len(train_data[train_data['label'] == 'Personal Loan']),
                'Vehicle loan or lease': len(train_data[train_data['label'] == 'Vehicle loan or lease']),
                'Other financial service': len(train_data[train_data['label'] == 'Other financial service'])  # Keep 105 examples for this class
            }
rus = RandomUnderSampler(sampling_strategy = desired_samples_train,random_state=42)
X_train_res, y_train_res = rus.fit_resample(train_data[['text']], train_data['label'])
X_train_res = X_train_res['text']

In [ ]:
test_data = pd.DataFrame({'text': X_test, 'label': y_test})
            desired_samples_test = {
                'Credit Reporting/Repair Services/Consumer Reports': 32302,
                'Debt Collection': len(test_data[test_data['label'] == 'Debt Collection']),
                'Credit/Prepaid Cards': len(test_data[test_data['label'] == 'Credit/Prepaid Cards']),
                'Mortgage': len(test_data[test_data['label'] == 'Mortgage']),
                'Bank Account Services': len(test_data[test_data['label'] == 'Bank Account Services']),
                'Student loan': len(test_data[test_data['label'] == 'Student loan']),
                'Money Services/Transfer': len(test_data[test_data['label'] == 'Money Services/Transfer']),
                'Personal Loan': len(test_data[test_data['label'] == 'Personal Loan']),
                'Vehicle loan or lease': len(test_data[test_data['label'] == 'Vehicle loan or lease']),
                'Other financial service': len(test_data[test_data['label'] == 'Other financial service'])  # Keep 105 examples for this class
            }
rus = RandomUnderSampler(sampling_strategy = desired_samples_test,random_state=42)
X_test_res, y_test_res = rus.fit_resample(test_data[['text']], test_data['label'])
X_test_res = X_test_res['text']

## Downsampling is done lets preprocess the data

In [ ]:
def preprocess_text(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\d", " ", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub("\[.*?\]", "", text)
    text = re.sub(r"""[.,/""/'':-]""", '', text) #Removes special character from the text
    text = re.sub(r'xxxx', '', text) # Removes xxxx pattern from the text
    text = re.sub(r'[\{\}\$]', '', text)  # Remove curly braces and dollar signs
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = text.lower()
    return text

In [ ]:
X_train_res= X_train_res.apply(preprocess_text)
X_test_res= X_test_res.apply(preprocess_text)

##Lets Vectorize the data and label encode the data

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        stop_words='english')

X_train_tfidf = tfidf.fit_transform(X_train_res)
X_test_tfidf = tfidf.transform(X_test_res)

logger.info("Converted text data into numerical vector representation using TF-IDF")
logger.info("Converting ylabels to classes using LabelEncoder")

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_res)
y_test_encoded = le.transform(y_test_res)

#OUR DATA TRANSFORMATION HAS BEEN COMPLETED